# Chapter 41: TypeScript Advanced

TypeScript's type system becomes exponentially more powerful when leveraged for full-stack Next.js applications. Beyond basic interfaces and generics, advanced patterns enable compile-time guarantees that eliminate entire categories of runtime errors—from API contract mismatches to database query vulnerabilities. Mastering these patterns allows you to build self-documenting codebases where the compiler enforces architectural constraints, making refactoring large applications safer and faster.

By the end of this chapter, you'll master implementing type-safe API routes with generic handlers, creating branded types for entity ID discrimination, building conditional component props with discriminated unions, optimizing type performance for large codebases, implementing type guards for runtime validation, and constructing type-level middleware for cross-cutting concerns.

## 41.1 Type-Safe API Routes

Enforce contracts between client and server with generic route handlers and typed fetch wrappers.

### Generic Route Handler Pattern

```typescript
// lib/api/types.ts
import { NextRequest, NextResponse } from 'next/server';
import { z } from 'zod';

// HTTP Method types
type HttpMethod = 'GET' | 'POST' | 'PUT' | 'PATCH' | 'DELETE';

// Generic route handler type
export type TypedRouteHandler<
  TParams = unknown,
  TBody = unknown,
  TQuery = unknown,
  TResponse = unknown
> = (
  request: NextRequest,
  context: {
    params: TParams;
    body: TBody;
    query: TQuery;
  }
) => Promise<NextResponse<TResponse>>;

// Route configuration with validation
interface RouteConfig<TBody, TQuery, TResponse> {
  method: HttpMethod;
  bodySchema?: z.ZodSchema<TBody>;
  querySchema?: z.ZodSchema<TQuery>;
  handler: TypedRouteHandler<any, TBody, TQuery, TResponse>;
}

// Type-safe route builder
export function createRoute<TBody, TQuery, TResponse>(
  config: RouteConfig<TBody, TQuery, TResponse>
) {
  return async (request: NextRequest, { params }: { params: any }) => {
    try {
      // Validate query params
      let query: TQuery = {} as TQuery;
      if (config.querySchema) {
        const { searchParams } = new URL(request.url);
        const queryObj = Object.fromEntries(searchParams);
        query = config.querySchema.parse(queryObj);
      }

      // Validate body
      let body: TBody = {} as TBody;
      if (config.bodySchema && ['POST', 'PUT', 'PATCH'].includes(config.method)) {
        const json = await request.json();
        body = config.bodySchema.parse(json);
      }

      // Execute handler with typed context
      return await config.handler(request, { params, body, query });
    } catch (error) {
      if (error instanceof z.ZodError) {
        return NextResponse.json(
          { error: 'Validation failed', details: error.errors },
          { status: 400 }
        );
      }
      throw error;
    }
  };
}

// Usage example
// app/api/users/route.ts
import { z } from 'zod';
import { createRoute } from '@/lib/api/types';

const createUserSchema = z.object({
  name: z.string().min(1),
  email: z.string().email(),
  role: z.enum(['admin', 'user']).default('user'),
});

const querySchema = z.object({
  page: z.coerce.number().default(1),
  limit: z.coerce.number().default(10),
});

type CreateUserBody = z.infer<typeof createUserSchema>;
type ListUsersQuery = z.infer<typeof querySchema>;
type UserResponse = { id: string; name: string; email: string };

// POST /api/users - Type-safe creation
export const POST = createRoute<CreateUserBody, never, UserResponse>({
  method: 'POST',
  bodySchema: createUserSchema,
  handler: async (req, { body }) => {
    // 'body' is fully typed as CreateUserBody
    const user = await db.user.create({ data: body });
    
    return NextResponse.json({
      id: user.id,
      name: user.name,
      email: user.email,
    });
  },
});

// GET /api/users - Type-safe listing
export const GET = createRoute<never, ListUsersQuery, UserResponse[]>({
  method: 'GET',
  querySchema: querySchema,
  handler: async (req, { query }) => {
    // 'query' is typed with defaults applied
    const users = await db.user.findMany({
      skip: (query.page - 1) * query.limit,
      take: query.limit,
    });
    
    return NextResponse.json(users);
  },
});
```

### Type-Safe Fetch Client

```typescript
// lib/api/client.ts
type ApiResponse<T> = 
  | { success: true; data: T }
  | { success: false; error: string; details?: any };

// Route definitions map
interface ApiRoutes {
  '/api/users': {
    GET: {
      query: { page?: number; limit?: number };
      response: Array<{ id: string; name: string; email: string }>;
    };
    POST: {
      body: { name: string; email: string; role?: 'admin' | 'user' };
      response: { id: string; name: string; email: string };
    };
  };
  '/api/users/[id]': {
    GET: {
      params: { id: string };
      response: { id: string; name: string; email: string; posts: number };
    };
    PUT: {
      params: { id: string };
      body: { name?: string; email?: string };
      response: { id: string; name: string; email: string };
    };
  };
}

type RouteMethod<T extends keyof ApiRoutes, M extends keyof ApiRoutes[T]> = 
  ApiRoutes[T][M] extends { method: M } ? ApiRoutes[T][M] : never;

// Typed fetch function
export async function apiFetch<
  TPath extends keyof ApiRoutes,
  TMethod extends keyof ApiRoutes[TPath]
>(
  path: TPath,
  method: TMethod,
  options: {
    params?: TPath extends `${string}[${string}]${string}` 
      ? ApiRoutes[TPath][TMethod] extends { params: infer P } ? P : never 
      : never;
    body?: ApiRoutes[TPath][TMethod] extends { body: infer B } ? B : never;
    query?: ApiRoutes[TPath][TMethod] extends { query: infer Q } ? Q : never;
  } = {}
): Promise<ApiResponse<ApiRoutes[TPath][TMethod] extends { response: infer R } ? R : never>> {
  // Build URL with params
  let url = path as string;
  if (options.params) {
    Object.entries(options.params).forEach(([key, value]) => {
      url = url.replace(`[${key}]`, value as string);
    });
  }

  // Add query params
  if (options.query) {
    const params = new URLSearchParams();
    Object.entries(options.query).forEach(([key, value]) => {
      if (value !== undefined) params.append(key, String(value));
    });
    url += `?${params.toString()}`;
  }

  const response = await fetch(url, {
    method: method as string,
    headers: options.body ? { 'Content-Type': 'application/json' } : undefined,
    body: options.body ? JSON.stringify(options.body) : undefined,
  });

  const data = await response.json();
  return data;
}

// Usage
async function example() {
  // Fully typed autocomplete
  const users = await apiFetch('/api/users', 'GET', {
    query: { page: 1, limit: 10 }
  });
  
  if (users.success) {
    users.data; // Array<{ id, name, email }>
  }

  const user = await apiFetch('/api/users/[id]', 'GET', {
    params: { id: '123' }
  });
}
```

## 41.2 Branded Types for Entity Safety

Prevent ID confusion between different entity types using branded types and nominal typing.

### Branded Type Implementation

```typescript
// lib/types/branded.ts
declare const brand: unique symbol;

export type Brand<T, TBrand> = T & { [brand]: TBrand };

// Entity ID types
export type UserId = Brand<string, 'UserId'>;
export type PostId = Brand<string, 'PostId'>;
export type OrganizationId = Brand<string, 'OrganizationId'>;

// Factory functions with validation
export function createUserId(id: string): UserId {
  if (!id.startsWith('user_')) {
    throw new Error('Invalid UserId format');
  }
  return id as UserId;
}

export function createPostId(id: string): PostId {
  if (!id.startsWith('post_')) {
    throw new Error('Invalid PostId format');
  }
  return id as PostId;
}

// Database layer with branded types
// lib/db/types.ts
import { Brand } from './branded';

export interface DatabaseUser {
  id: UserId;
  email: string;
  organizationId: OrganizationId;
}

export interface DatabasePost {
  id: PostId;
  authorId: UserId; // Cannot accidentally assign OrganizationId
  title: string;
}

// Type-safe repository pattern
export class UserRepository {
  async findById(id: UserId): Promise<DatabaseUser | null> {
    // Implementation
    return db.user.findUnique({ where: { id } });
  }
  
  // Compile error if you try to pass PostId
  async delete(id: UserId): Promise<void> {
    await db.user.delete({ where: { id } });
  }
}

// Usage
const userId = createUserId('user_123');
const postId = createPostId('post_456');

const repo = new UserRepository();
await repo.findById(userId); // OK
await repo.findById(postId); // Compile error: Argument of type 'PostId' is not assignable to parameter of type 'UserId'
```

### Type-Safe Relations

```typescript
// lib/types/relations.ts
type EntityWithId<T extends string> = { id: Brand<string, T> };

// Type-safe relation loader
export async function loadRelation<
  TFrom extends EntityWithId<any>,
  TTo extends EntityWithId<any>
>(
  from: TFrom,
  relation: {
    foreignKey: keyof TFrom;
    target: { findMany: (args: { where: any }) => Promise<TTo[]> };
  }
): Promise<TTo[]> {
  return await relation.target.findMany({
    where: { [relation.foreignKey]: from.id }
  });
}

// Usage
const user = await db.user.findFirst();
if (user) {
  // Type-safe relation loading
  const posts = await loadRelation(user, {
    foreignKey: 'authorId',
    target: db.post
  });
}
```

## 41.3 Conditional Component Props

Build flexible components with props that change based on configuration using discriminated unions.

### Polymorphic Component Pattern

```typescript
// components/ui/button.tsx
import { ButtonHTMLAttributes, ReactNode } from 'react';
import Link from 'next/link';

type BaseProps = {
  children: ReactNode;
  variant?: 'primary' | 'secondary' | 'ghost';
  size?: 'sm' | 'md' | 'lg';
  isLoading?: boolean;
};

// Discriminated union for different behaviors
type ButtonAsButton = BaseProps & {
  as?: 'button';
  onClick?: () => void;
  type?: 'button' | 'submit' | 'reset';
  disabled?: boolean;
} & Omit<ButtonHTMLAttributes<HTMLButtonElement>, keyof BaseProps>;

type ButtonAsLink = BaseProps & {
  as: 'link';
  href: string;
  prefetch?: boolean;
  replace?: boolean;
} & Omit<React.ComponentProps<typeof Link>, keyof BaseProps | 'as'>;

type ButtonAsExternal = BaseProps & {
  as: 'external';
  href: string;
  target?: string;
  rel?: string;
} & Omit<React.AnchorHTMLAttributes<HTMLAnchorElement>, keyof BaseProps | 'as'>;

type ButtonProps = ButtonAsButton | ButtonAsLink | ButtonAsExternal;

export function Button(props: ButtonProps) {
  const { children, variant = 'primary', size = 'md', isLoading, ...rest } = props;
  
  const className = `btn btn-${variant} btn-${size} ${isLoading ? 'loading' : ''}`;
  
  if (rest.as === 'link') {
    const { as, ...linkProps } = rest;
    return <Link className={className} {...linkProps}>{children}</Link>;
  }
  
  if (rest.as === 'external') {
    const { as, ...anchorProps } = rest;
    return (
      <a className={className} target="_blank" rel="noopener noreferrer" {...anchorProps}>
        {children}
      </a>
    );
  }
  
  // Default button
  const { as, ...buttonProps } = rest;
  return <button className={className} {...buttonProps}>{children}</button>;
}

// Usage with full type safety
<Button onClick={() => console.log('clicked')}>Click me</Button>
<Button as="link" href="/dashboard" prefetch>Go to Dashboard</Button>
<Button as="external" href="https://example.com">External Link</Button>
// <Button as="link" onClick={() => {}}>Error: onClick not valid for link</Button>
```

### Form Field Type Safety

```typescript
// components/form/types.ts
import { InputHTMLAttributes, SelectHTMLAttributes } from 'react';

type FieldType = 'text' | 'email' | 'number' | 'select' | 'textarea';

type BaseFieldProps<T> = {
  name: string;
  label: string;
  error?: string;
  required?: boolean;
};

type TextFieldProps = BaseFieldProps<string> & {
  type: 'text' | 'email';
  minLength?: number;
  maxLength?: number;
  pattern?: string;
} & Omit<InputHTMLAttributes<HTMLInputElement>, 'type' | 'name'>;

type NumberFieldProps = BaseFieldProps<number> & {
  type: 'number';
  min?: number;
  max?: number;
  step?: number;
} & Omit<InputHTMLAttributes<HTMLInputElement>, 'type' | 'name'>;

type SelectFieldProps<T extends string> = BaseFieldProps<T> & {
  type: 'select';
  options: Array<{ value: T; label: string }>;
} & Omit<SelectHTMLAttributes<HTMLSelectElement>, 'name'>;

type TextareaFieldProps = BaseFieldProps<string> & {
  type: 'textarea';
  rows?: number;
} & Omit<React.TextareaHTMLAttributes<HTMLTextAreaElement>, 'name'>;

export type FormFieldProps = 
  | TextFieldProps 
  | NumberFieldProps 
  | SelectFieldProps<string> 
  | TextareaFieldProps;

// Type guard for narrowing
export function isNumberField(props: FormFieldProps): props is NumberFieldProps {
  return props.type === 'number';
}
```

## 41.4 Type Performance Optimization

Prevent TypeScript compiler slowdowns in large Next.js applications with proper type constraints.

### Recursive Type Limits

```typescript
// lib/types/optimized.ts
// BAD: Deep recursive types cause exponential compilation
type DeepNested<T> = T extends object 
  ? { [K in keyof T]: DeepNested<T[K]> } 
  : T;

// GOOD: Use depth limiting
type DeepNestedLimited<T, Depth extends number = 5> = [Depth] extends [never]
  ? T
  : T extends object
  ? { [K in keyof T]: DeepNestedLimited<T[K], Prev<Depth>> }
  : T;

type Prev<T extends number> = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10][T];

// Usage
interface ComplexData {
  level1: {
    level2: {
      level3: {
        level4: { value: string };
      };
    };
  };
}

type Limited = DeepNestedLimited<ComplexData, 3>; // Stops at level 3
```

### String Literal Unions vs Enums

```typescript
// lib/types/performance.ts
// BAD for large sets: String literal unions create huge union types
type Color = 'red' | 'blue' | 'green' | /* ... 100 more */ ;

// GOOD: Use const assertion with objects for large sets
export const Colors = {
  red: '#FF0000',
  blue: '#0000FF',
  green: '#00FF00',
  // ... 100 more
} as const;

export type Color = keyof typeof Colors;

// Access values efficiently
export function getColorHex(color: Color): string {
  return Colors[color];
}
```

### Generic Constraints

```typescript
// lib/types/constraints.ts
// BAD: Unconstrained generics allow invalid types
function processData<T>(data: T): T {
  return data.toString(); // Error if T doesn't have toString
}

// GOOD: Constrain to known shapes
interface Stringifiable {
  toString(): string;
}

function processDataSafe<T extends Stringifiable>(data: T): string {
  return data.toString();
}

// Use with database models
interface Model {
  id: string;
  toJSON(): object;
}

function serializeModel<T extends Model>(model: T): string {
  return JSON.stringify(model.toJSON());
}
```

## 41.5 Type Guards and Assertion Functions

Implement runtime validation with TypeScript type guards for boundary safety.

### Zod Integration with Type Guards

```typescript
// lib/validation/guards.ts
import { z } from 'zod';

// Environment variable validation with type narrowing
const envSchema = z.object({
  NODE_ENV: z.enum(['development', 'production', 'test']),
  DATABASE_URL: z.string().url(),
  API_KEY: z.string().min(32),
});

export type ValidatedEnv = z.infer<typeof envSchema>;

export function validateEnv(): asserts env is ValidatedEnv {
  const parsed = envSchema.safeParse(process.env);
  
  if (!parsed.success) {
    const errors = parsed.error.errors.map(e => `${e.path}: ${e.message}`).join('\n');
    throw new Error(`Invalid environment variables:\n${errors}`);
  }
}

// Usage at application startup
validateEnv(); // Throws if invalid, narrows type if valid
// Now process.env is typed as ValidatedEnv
```

### API Response Type Guards

```typescript
// lib/api/guards.ts
interface ApiSuccess<T> {
  success: true;
  data: T;
}

interface ApiError {
  success: false;
  error: string;
  code: number;
}

type ApiResponse<T> = ApiSuccess<T> | ApiError;

// Type guard function
export function isApiSuccess<T>(response: ApiResponse<T>): response is ApiSuccess<T> {
  return response.success === true;
}

export function isApiError<T>(response: ApiResponse<T>): response is ApiError {
  return response.success === false;
}

// Usage in data fetching
const response = await fetch('/api/user').then(r => r.json()) as ApiResponse<User>;

if (isApiSuccess(response)) {
  response.data; // Typed as User
} else {
  response.error; // Typed as string
  response.code;  // Typed as number
}
```

## Key Takeaways from Chapter 41

1. **Generic Route Handlers**: Create `createRoute` higher-order functions that accept Zod schemas and return typed Next.js route handlers. This enforces request/response contracts at compile time while providing runtime validation, eliminating the need for separate TypeScript interfaces and validation logic.

2. **Branded Types**: Use intersection types with unique symbols (`type UserId = string & { [brand]: 'UserId' }`) to prevent mixing ID types between entities. This catches bugs like passing a PostId to a function expecting UserId at compile time, even though both are underlying strings.

3. **Discriminated Unions**: Design component props as discriminated unions with an `as` property ('button' | 'link' | 'external') to create polymorphic components with mutually exclusive props. TypeScript narrows the available props based on the `as` value, preventing invalid combinations like `onClick` on anchor elements.

4. **Recursive Depth Limiting**: Constrain recursive type definitions with depth counters (`Depth extends number = 5`) to prevent TypeScript compiler performance degradation on deeply nested objects. Use tail-recursive techniques and prefer interfaces over type aliases for large object shapes.

5. **Assertion Functions**: Use `asserts param is Type` return annotations for validation functions that throw on failure. After calling `validateEnv()`, TypeScript narrows `process.env` to the validated type without manual casting, enabling dot-autocomplete on environment variables.

6. **Const Assertions**: Apply `as const` to configuration objects instead of enums for better tree-shaking and type inference. Access object keys with `keyof typeof` to derive union types that stay synchronized with runtime values.

7. **Type Guard Patterns**: Implement `isApiSuccess(response)` predicates to narrow union types (success | error) after API calls. This eliminates manual casting and provides IntelliSense for the correct branch properties within conditional blocks.

## Coming Up Next

**Chapter 42: Migrating to Next.js**

With advanced TypeScript patterns mastered, it's time to address the practical challenge of bringing existing applications into the Next.js ecosystem. In Chapter 42, we'll explore migration strategies from Create React App, Gatsby, Vue/Nuxt, and plain React applications. You'll learn incremental adoption techniques, feature-by-feature migration approaches, handling routing differences, and maintaining SEO rankings during transitions.